In [2]:
''' This code follows the PINN - Navier Stokes project done by Computational Domain, but adapted for our secific scenarios'''

import torch
import torch.nn as nn
import numpy as np
import scipy.io
from matplotlib import pyplot as plt

nu = 0.01
batch_size = 1000  # Define a batch size to control memory usage

# Load data and create training dataset
data = scipy.io.loadmat('cylinder_wake.mat')

U_star = data['U_star']  # N x 2 x T
P_star = data['p_star']  # N x T
t_star = data['t']  # T x 1
X_star = data['X_star']  # N x 2

N = X_star.shape[0]
T = t_star.shape[0]

# Split data into training (first 70%) and testing (last 30%) sets based on timesteps
train_T = int(0.7 * T)
test_T = T - train_T

# Rearrange Data for Training
XX_train = np.tile(X_star[:, 0:1], (1, train_T))  # N x train_T
YY_train = np.tile(X_star[:, 1:2], (1, train_T))  # N x train_T
TT_train = np.tile(t_star[:train_T], (1, N)).T  # N x train_T

UU_train = U_star[:, 0, :train_T]  # N x train_T
VV_train = U_star[:, 1, :train_T]  # N x train_T
PP_train = P_star[:, :train_T]  # N x train_T

x_train = torch.tensor(XX_train.flatten()[:, None], dtype=torch.float32, requires_grad=True)
y_train = torch.tensor(YY_train.flatten()[:, None], dtype=torch.float32, requires_grad=True)
t_train = torch.tensor(TT_train.flatten()[:, None], dtype=torch.float32, requires_grad=True)

u_train = torch.tensor(UU_train.flatten()[:, None], dtype=torch.float32)
v_train = torch.tensor(VV_train.flatten()[:, None], dtype=torch.float32)
p_train = torch.tensor(PP_train.flatten()[:, None], dtype=torch.float32)

# Rearrange Data for Testing
XX_test = np.tile(X_star[:, 0:1], (1, test_T))  # N x test_T
YY_test = np.tile(X_star[:, 1:2], (1, test_T))  # N x test_T
TT_test = np.tile(t_star[train_T:], (1, N)).T  # N x test_T

UU_test = U_star[:, 0, train_T:]  # N x test_T
VV_test = U_star[:, 1, train_T:]  # N x test_T
PP_test = P_star[:, train_T:]  # N x test_T

x_test = torch.tensor(XX_test.flatten()[:, None], dtype=torch.float32, requires_grad=True)
y_test = torch.tensor(YY_test.flatten()[:, None], dtype=torch.float32, requires_grad=True)
t_test = torch.tensor(TT_test.flatten()[:, None], dtype=torch.float32, requires_grad=True)

u_test = torch.tensor(UU_test.flatten()[:, None], dtype=torch.float32)
v_test = torch.tensor(VV_test.flatten()[:, None], dtype=torch.float32)
p_test = torch.tensor(PP_test.flatten()[:, None], dtype=torch.float32)

# Define a simple ResNet block
class ResNetBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super(ResNetBlock, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
        self.activation = nn.Tanh()
    
    def forward(self, x):
        return self.activation(self.linear(x)) + x

# Define the ResNet-style neural network with 7 hidden layers
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.input_layer = nn.Linear(3, 20)
        self.res_block1 = ResNetBlock(20, 20)
        self.res_block2 = ResNetBlock(20, 20)
        self.res_block3 = ResNetBlock(20, 20)
        self.res_block4 = ResNetBlock(20, 20)
        self.res_block5 = ResNetBlock(20, 20)
        self.res_block6 = ResNetBlock(20, 20)
        self.res_block7 = ResNetBlock(20, 20)
        self.output_layer = nn.Linear(20, 3)
        self.activation = nn.Tanh()
    
    def forward(self, x):
        x = self.activation(self.input_layer(x))
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.res_block3(x)
        x = self.res_block4(x)
        x = self.res_block5(x)
        x = self.res_block6(x)
        x = self.res_block7(x)
        x = self.output_layer(x)
        return x

net = ResNet()

# Define the loss function
mse = nn.MSELoss()

# Function written by Computational Domain
def create_pde(x, y, t):
    res = net(torch.hstack((x, y, t)))
    psi, p = res[:, 0:1], res[:, 1:2]

    u = torch.autograd.grad(psi, y, grad_outputs=torch.ones_like(psi), create_graph=True)[0]  
    v = -1. * torch.autograd.grad(psi, x, grad_outputs=torch.ones_like(psi), create_graph=True)[0]

    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
    u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]

    v_x = torch.autograd.grad(v, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(v_x), create_graph=True)[0]
    v_y = torch.autograd.grad(v, y, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_yy = torch.autograd.grad(v_y, y, grad_outputs=torch.ones_like(v_y), create_graph=True)[0]
    v_t = torch.autograd.grad(v, t, grad_outputs=torch.ones_like(v), create_graph=True)[0]

    p_x = torch.autograd.grad(p, x, grad_outputs=torch.ones_like(p), create_graph=True)[0]
    p_y = torch.autograd.grad(p, y, grad_outputs=torch.ones_like(p), create_graph=True)[0]

    f = u_t + u * u_x + v * u_y + p_x - nu * (u_xx + u_yy)
    g = v_t + u * v_x + v * v_y + p_y - nu * (v_xx + v_yy)

    return u, v, p, f, g

# Use LBFGS optimizer with early stopping
optimizer_lbfgs = torch.optim.LBFGS(net.parameters(), lr=1, max_iter=20000, max_eval=50000,
                                    history_size=50, tolerance_grad=1e-05, tolerance_change=0.5 * np.finfo(float).eps,
                                    line_search_fn="strong_wolfe")

loss_history = []
f_residual_history = []
g_residual_history = []
lbfgs_iter = 0  # Initialize the LBFGS iteration counter
patience = 200  # Number of iterations with no significant improvement before stopping
min_delta = 1e-4  # Minimum change to qualify as an improvement
best_loss = float('inf')
patience_counter = 0

# Fine-tuning with LBFGS and early stopping in batches
for i in range(0, x_train.shape[0], batch_size):
    x_batch = x_train[i:i+batch_size].detach().clone().requires_grad_(True)
    y_batch = y_train[i:i+batch_size].detach().clone().requires_grad_(True)
    t_batch = t_train[i:i+batch_size].detach().clone().requires_grad_(True)
    u_batch = u_train[i:i+batch_size]
    v_batch = v_train[i:i+batch_size]
    p_batch = p_train[i:i+batch_size]
    
    def closure():
        global lbfgs_iter, best_loss, patience_counter  # Access the global iteration variables
        optimizer_lbfgs.zero_grad()

        u_prediction, v_prediction, p_prediction, f_prediction, g_prediction = create_pde(x_batch, y_batch, t_batch)
        u_loss = mse(u_prediction, u_batch)
        v_loss = mse(v_prediction, v_batch)
        p_loss = mse(p_prediction, p_batch)
        f_loss = mse(f_prediction, torch.zeros_like(f_prediction))
        g_loss = mse(g_prediction, torch.zeros_like(g_prediction))

        loss = u_loss + v_loss + p_loss + f_loss + g_loss
        loss.backward()

        # Print the current iteration and loss
        lbfgs_iter += 1
        if lbfgs_iter % 10 == 0:
            print(f'LBFGS Iteration {lbfgs_iter}, Loss: {loss.item()}')

        # Early stopping logic
        if loss.item() < best_loss - min_delta:
            best_loss = loss.item()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered")
                return loss

        # Track loss and residuals
        loss_history.append(loss.item())
        f_residual_history.append(f_loss.item())
        g_residual_history.append(g_loss.item())

        return loss
    
    optimizer_lbfgs.step(closure)

# Plot the loss curve
plt.figure()
plt.plot(loss_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Loss Curve')
plt.show()

# Plot f and g residuals
plt.figure()
plt.plot(f_residual_history, label='f Residual')
plt.plot(g_residual_history, label='g Residual')
plt.xlabel('Iteration')
plt.ylabel('Residual')
plt.title('f and g Residuals Curve')
plt.legend()
plt.show()

# Save the trained model
torch.save(net.state_dict(), 'pinn_70_30_resnet_lbfgs.pt')

# Load the trained model and evaluate
net.eval()

u_out, v_out, p_out, f_out, g_out = create_pde(x_test, y_test, t_test)



c:\Users\zhyy2\anaconda3\Lib\site-packages\torch\autograd\graph.py:744: UserWarning: grad and param do not obey the gradient layout contract. This is not an error, but may impair performance.
grad.sizes() = [1000, 1], strides() = [1, 0]
param.sizes() = [1000, 1], strides() = [1, 0] (Triggered internally at ..\torch/csrc/autograd/functions/accumulate_grad.h:219.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


LBFGS Iteration 10, Loss: 0.42084017395973206
LBFGS Iteration 20, Loss: 0.010974712669849396
LBFGS Iteration 30, Loss: 0.00333099951967597
LBFGS Iteration 40, Loss: 0.001702727866359055
LBFGS Iteration 50, Loss: 0.0011226300848647952
LBFGS Iteration 60, Loss: 0.0009786197915673256
LBFGS Iteration 70, Loss: 0.0008482502307742834
LBFGS Iteration 80, Loss: 0.0007915855967439711
LBFGS Iteration 90, Loss: 0.0006918628350831568
LBFGS Iteration 100, Loss: 0.0006351459305733442
LBFGS Iteration 110, Loss: 0.0006008865311741829


KeyboardInterrupt: 